# gmsh2netgen

Convert a simple 2D Gmsh geometry into a mesh, export it, and visualize via PyVista. This notebook demonstrates geometry construction, physical groups, meshing, and conversion to a VTK data structure.

## 1. Imports and utilities
Import Gmsh, a helper module `utils` (providing `gmsh2VTU`), and PyVista for visualization.

In [1]:
import gmsh
import utils as ut
import pyvista as pv
from ngsolve import *
from netgen.geom2d import SplineGeometry
from xfem import *
from math import pi
from xfem.lset_spacetime import *
ngsglobals.msg_level = 1

#import netgen.gui 
DrawDC = MakeDiscontinuousDraw(Draw)

importing ngsxfem-2.1.2504


## 3. Build and mesh geometry
We now create the CAD entities, define a curve loop and surface, then set physical groups before generating a 2D mesh.

## 3. Structured (transfinite) mesh generation
We build a 4-sided geometry and impose a structured mesh via transfinite interpolation on curves and the surface. Optionally we recombine triangles into quads.

### Parameters
Rectangle defined by:
- `Lh`: horizontal length (x-direction)
- `Lv`: vertical length (y-direction)

Structured discretization:
- `Nh`: number of intervals along horizontal edges (bottom & top)
- `Nv`: number of intervals along vertical edges (left & right)
- Nodes per edge = intervals + 1

Grading:
- `progh`: grading factor for horizontal edges (>1 clusters towards the second point of each edge)
- `progv`: grading factor for vertical edges

Other:
- `recombine`: request quad recombination
- `order`: element order (1 = linear)


In [2]:
# Geometry lengths
Lh = 1.2
Lv = 2.0

# Mesh divisions
Nh = 18
Nv = 30

# Grading factors
progh = 1.0
progv = 1.0

recombine = True  # request quads
order = 1         # element order


In [3]:
gmsh.initialize()
model = gmsh.model
occ = model.occ
model.add("structured_rect")

occ.addPoint(-Lh/2, -Lv/2, 0.0)
occ.addPoint( Lh/2, -Lv/2, 0.0)
occ.addPoint( Lh/2,  Lv/2, 0.0)
occ.addPoint(-Lh/2,  Lv/2, 0.0)

occ.addLine(1, 2, 1)
occ.addLine(2, 3, 2)
occ.addLine(3, 4, 3)
occ.addLine(4, 1, 4)

occ.addCurveLoop([1,2,3,4], 1)
occ.addPlaneSurface([1], 1)
occ.synchronize()

model.mesh.setTransfiniteCurve(1, Nh+1, coef=progh)
model.mesh.setTransfiniteCurve(3, Nh+1, coef=progh)
model.mesh.setTransfiniteCurve(2, Nv+1, coef=progv)
model.mesh.setTransfiniteCurve(4, Nv+1, coef=progv)

model.mesh.setTransfiniteSurface(1, cornerTags=[1,2,3,4])
if recombine:
    model.mesh.setRecombine(2, 1)

model.addPhysicalGroup(1, [1,2,3,4], 101)
model.setPhysicalName(1,101,"boundary")
model.addPhysicalGroup(2, [1], 201)
model.setPhysicalName(2,201,"domain")

model.mesh.generate(2)
model.mesh.setOrder(order)
model.mesh.optimize("Netgen")

unstructuredGrid = ut.gmsh2VTU(model)  # keep model active for translation step


Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 60%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.00028289s, CPU 0.000322s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Transfinite)
Info    : Done meshing 2D (Wall 0.000519568s, CPU 0.000853s)
Info    : 589 nodes 640 elements
Info    : Optimizing mesh (Netgen)...
Info    : Done optimizing mesh (Wall 4.398e-06s, CPU 4e-06s)
Element types: 3


### 4. Visualize
Wrap the Gmsh mesh as a PyVista unstructured grid and plot with edge overlay.

In [4]:
plotter = pv.Plotter()
actor = plotter.add_mesh(pv.wrap(unstructuredGrid), scalars="Region", cmap="rainbow")
plotter.view_xy()

actor.prop.show_edges = True
plotter.show()

Widget(value='<iframe src="http://localhost:34417/index.html?ui=P_0x74ca30dec290_0&reconnect=auto" class="pyvi…

### 5. Convert Gmsh model to Netgen mesh
We extract node coordinates and element connectivities from the live Gmsh model and build an equivalent Netgen 2D mesh using low-level Netgen Python API.

In [5]:
from netgen.meshing import Mesh as NGMesh, MeshPoint, Element2D, Element1D, FaceDescriptor
from netgen.csg import Pnt
import ngsolve
from ngsolve import Mesh as NGSolveMesh

# Build a Netgen (geometry) mesh first
ngmesh = NGMesh()
ngmesh.dim = 2

# FaceDescriptor: surface number 1 -> material / bc info (here domin=1, bc=1)
fd = ngmesh.Add(FaceDescriptor(surfnr=1, domin=1, bc=1))

nodeTags, coords, _ = gmsh.model.mesh.getNodes()

# Build mapping gmsh tag -> netgen point number
ptMap = {}
for i, tag in enumerate(nodeTags):
    x = coords[3*i]; y = coords[3*i+1]; z = coords[3*i+2]
    pnum = ngmesh.Add(MeshPoint(Pnt(x,y,z)))
    ptMap[tag] = pnum

# Elements (2D)
all2d = gmsh.model.mesh.getElements(2)
for elemType, elemTags, elemNodeTags in zip(*all2d):
    # elemType 2: triangle (3 nodes), elemType 3: quadrangle (4 nodes)
    nn = {2:3, 3:4}.get(elemType)
    if nn is None:
        continue
    for e in range(len(elemTags)):
        nodes = elemNodeTags[nn*e:nn*(e+1)]
        ngNodes = [ptMap[t] for t in nodes]
        ngmesh.Add(Element2D(fd, ngNodes))

# Elements (1D) (boundary)
edge_types = gmsh.model.mesh.getElements(1)
for elemType, elemTags, elemNodeTags in zip(*edge_types):
    if elemType != 1:  # only 2-node lines
        continue
    for e in range(len(elemTags)):
        n1, n2 = elemNodeTags[2*e:2*e+2]
        ngmesh.Add(Element1D([ptMap[n1], ptMap[n2]], index=1))

# Wrap as ngsolve Mesh and reuse variable name 'mesh' expected by later cells
mesh = NGSolveMesh(ngmesh)
print("Converted to ngsolve Mesh ->", type(mesh))
print(f"Netgen/ngsolve mesh: {mesh.ne} elements, {mesh.nv} vertices")

from ngsolve.webgui import Draw
Draw(mesh)

Converted to ngsolve Mesh -> <class 'ngsolve.comp.Mesh'>
Netgen/ngsolve mesh: 540 elements, 589 vertices


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [6]:
from ngsolve import *
from xfem import *
from math import pi
from xfem.lset_spacetime import *
ngsglobals.msg_level = 1

#import netgen.gui 
DrawDC = MakeDiscontinuousDraw(Draw)

In [7]:
# DISCRETIZATION PARAMETERS:

# Parameter for refinement study:
i = 3
n_steps = 2**i
space_refs = i

# Polynomial order in time
k_t = 2
# Polynomial order in space
k_s = k_t
# Polynomial order in time for level set approximation
lset_order_time = k_t
# Integration order in time
time_order = 2 * k_t
# Time stepping parameters
tstart = 0
tend = 0.5
delta_t = (tend - tstart) / n_steps
maxh = 0.5
# Ghost-penalty parameter
gamma = 0.05

In [8]:
# Map from reference time to physical time
told = Parameter(tstart)
t = told + delta_t * tref
t.MakeVariable()

In [9]:
# Level set geometry
# Radius of disk (the geometry)
R = 0.5
# Position shift of the geometry in time
rho = (1 / (pi)) * sin(2 * pi * t)
# Convection velocity:
w = CoefficientFunction((0, rho.Diff(t)))
# Level set
r = sqrt(x**2 + (y - rho)**2)
levelset = r - R

# Diffusion coefficient
alpha = 1
# Solution
u_exact = cos(pi * r / R) * sin(pi * t)
# R.h.s.
coeff_f = (u_exact.Diff(t)
           - alpha * (u_exact.Diff(x).Diff(x) + u_exact.Diff(y).Diff(y))
           + w[0] * u_exact.Diff(x) + w[1] * u_exact.Diff(y)).Compile()

In [10]:
r_one_timestep = sqrt(x**2 + (y - (1 / (pi)) * sin(2 * pi * tref * tend))**2)
levelset_one_timestep = r_one_timestep - R
#TimeSlider_Draw(levelset_one_timestep,mesh,autoscale=False,min=-0.02,max=0.02,deformation=True)

from helper import ProjectOnMultiDimGF
print(type(mesh))
lset_to_show = ProjectOnMultiDimGF(levelset_one_timestep,mesh,order=3,sampling=8)
Draw(lset_to_show,mesh,"u_exact",autoscale=False,min=-0.5,max=1, interpolate_multidim=True, animate=True, deformation=True)

<class 'ngsolve.comp.Mesh'>


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [11]:
u_exact_one_timestep = cos(pi * r_one_timestep / R) * sin(pi * tref * tend)

from helper import ProjectPairOnMultiDimGF
lset_cf_to_show = ProjectPairOnMultiDimGF(levelset_one_timestep, u_exact_one_timestep,mesh,order=3,sampling=8)

Draw(lset_cf_to_show,mesh,"u_exact",eval_function="value.x>0.0?value.z:value.y",autoscale=False,
     min=-0.5,max=1, interpolate_multidim=True, animate=True)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

## Space-Time finite elements

* For the construction of a space-time `FESpace` we can combine any spatial `FESpace` with a scalar `FiniteElement` in a tensor-product fashion.
* Here, we use a nodal `FiniteElement` to simplify the extraction of spatial functions at fixed times.

In [12]:
# Spatial FESpace for solution
fes1 = H1(mesh, order=k_s, dgjumps=True)
# Time finite element (nodal!)
tfe = ScalarTimeFE(k_t)
# (Tensor product) space-time finite element space
st_fes = tfe * fes1

## Levelset mesh adaptation class

In order to achieve higher order accuracy in space and time, a variant of the isoparametric mapping for stationary domains is applied.

In [13]:
# Space time version of Levelset Mesh Adapation object. Also offers integrator
# helper functions that involve the correct mesh deformation
lsetadap = LevelSetMeshAdaptation_Spacetime(mesh, order_space=k_s,
                                            order_time=lset_order_time,
                                            threshold=0.5,
                                            discontinuous_qn=True)

### Space-Time version of the `CutInfo` class
The `CutInfo` class also works for space-time geometries. Its initialization is trivial:

In [14]:
ci = CutInfo(mesh, time_order=0)

Note the argument `time_order=0` which makes the `CutInfo` looking for space-time cut information afterwards.

In addition, we define a Bitarray for the facets of the mesh for later use in the definition of the Ghost-penalty stabilisation.

In [15]:
ba_facets = BitArray(mesh.nfacet)
active_dofs = BitArray(st_fes.ndof)

To Update the slab geometry later on (for updated values of `told`) we do the following:
 * update of the isoparametric mapping
 * update of the cut information
 * update of facet markers
 * update of dof markers

In [16]:
def UpdateTimeSlabGeometry():
    lsetadap.CalcDeformation(levelset)

    # Update markers in (space-time) mesh
    ci.Update(lsetadap.levelsetp1[INTERVAL], time_order=0)

    # re-compute the facets for stabilization:
    ba_facets[:] = GetFacetsWithNeighborTypes(mesh,
                                              a=ci.GetElementsOfType(HASNEG),
                                              b=ci.GetElementsOfType(IF))
    active_dofs[:] = GetDofsOfElements(st_fes, ci.GetElementsOfType(HASNEG))

Note that here the call of CalcDeformation of lsetadap entails the calculation of the P1 projection of the levelset function internally. The space-time P1-in-space level set approximation of `lsetadap` can be accessed by `lsetadap.levelsetp1[timetype]` where `timetype` is either `INTERVAL` which yields the space-time function or `TOP` or `BOTTOM` which yields the spatial P1 function that is obtained by restriction to either `tref=1` or `tref=0`. Similarly the access to the deformation is organized as `lsetadap.deformation[timetype]`.

### Solution GridFunction

In [17]:
gfu = GridFunction(st_fes)
u_last = CreateTimeRestrictedGF(gfu, 1)

## Variational formulation

Now we would like to derive a suitable variational formulation on the time slabs $Q^{n}$. 

We start by multiplying the equation  
\begin{equation*}
\partial_{t} u- \alpha \Delta{u} + w \cdot \nabla{u} = f \quad  in \quad \Omega(t),   \qquad  t \in [0,T] 
\end{equation*}
by a test function $v$ and perform integration by parts.

### Implementation of space-time integrals

In [18]:
u,v = st_fes.TnT()
h = specialcf.mesh_size

#### Transformation from reference interval to $(t_{n-1},t_n)$:
The transformation
$$
(x,\hat{t}) \to (x,t_{n-1} + \hat{t} \Delta t), \qquad v(x,t) = \hat{v}(x,\hat{t}), \quad u(x,t) = \hat{u}(x,\hat{t}).
$$
implies the following for the time derivative.

In [19]:
def dt(u):
    return 1.0 / delta_t * dtref(u)

Next, we define integration region symbols, which are the numerical counterparts of the regions introduced above. Note that the levelset deformation is included to achieve higher order in space. The `definedonelements` information is not necessary (for the first three symbols), but helpful for efficiency reasons.

In [20]:
dQ = delta_t * dCut(lsetadap.levelsetp1[INTERVAL], NEG, time_order=time_order,
                    deformation=lsetadap.deformation[INTERVAL],
                    definedonelements=ci.GetElementsOfType(HASNEG))
dOmold = dCut(lsetadap.levelsetp1[BOTTOM], NEG,
              deformation=lsetadap.deformation[BOTTOM],
              definedonelements=ci.GetElementsOfType(HASNEG), tref=0)
dOmnew = dCut(lsetadap.levelsetp1[TOP], NEG,
              deformation=lsetadap.deformation[TOP],
              definedonelements=ci.GetElementsOfType(HASNEG), tref=1)
dw = delta_t * dFacetPatch(definedonelements=ba_facets, time_order=time_order,
                           deformation=lsetadap.deformation[INTERVAL])

Now we setup the bilinear form and linear form corresponding to the previously described discrete variational formulation.

In [21]:
a = RestrictedBilinearForm(st_fes, "a", check_unused=False,
                           element_restriction=ci.GetElementsOfType(HASNEG),
                           facet_restriction=ba_facets)

First integral:  $(\partial_t u, v)_{Q^{n}}$

In [22]:
a += v * (dt(u) - dt(lsetadap.deform) * grad(u)) * dQ

Note that here, due to the time-dependent mesh deformation, the partial derivative in physical coordinates that we used in the formulation before corresponds to the partial derivative w.r.t. the reference configuration minus an additional mesh velocity contribution.

Second integral: $\alpha (\nabla{u},\nabla{v})_{Q^{n}}$

In [23]:
a += alpha * InnerProduct(grad(u), grad(v)) * dQ

Third integral: $(v, \nabla{u} \cdot w)_{Q^{n}}$

In [24]:
a += v * w * grad(u) * dQ

Fourth integral: $(u_{n−1}^+,v_{n−1}^+)_{\Omega^{n−1}}$

In [25]:
a += u * v * dOmold

Fifth integral:
$$ s_h(u,v) =   \sum\limits_{F \in F_{h}}{ \gamma_{j} \int\limits_{t_{n-1}}^{t_{n}}{   \int\limits_{\omega_F}{  h^{-2} [\![ u]\!] \, [\![ v]\!]         \, d\mathbf{x} \, dt.  }}} =   \sum\limits_{F \in F_{h}}{ \Delta t \ \gamma_{j} \int\limits_{t_{n-1}}^{t_{n}}{   \int\limits_{\omega_F}{  h^{-2} [\![ \hat u]\!] \, [\![ \hat v]\!]         \, d\mathbf{x} \, dt.  }}}  $$

In [26]:
a += h**(-2) * (1 + delta_t / h) * gamma * \
    (u - u.Other()) * (v - v.Other()) * dw

Sixth integral: $(f,v)_{Q^{n}}$

In [27]:
f = LinearForm(st_fes)
f += coeff_f * v * dQ

Seventh integral: $(u_{n−1}^-,v_{n−1}^+)_{\Omega^{n−1}}$

In [28]:
f += u_last * v * dOmold

### Solution of linear systems in every time step
* setup the new linear system
* solve the linear system

In [29]:
def SolveForTimeSlab():
    a.Assemble(reallocate=True)
    f.Assemble()
    inv = a.mat.Inverse(active_dofs)
    gfu.vec.data = inv * f.vec.data

### At the end of every time step, we
* store the solution at $t_n$ into a (purely) spatial `GridFunction` (to be used in next time step)
* compute the error
* update visualization

In [30]:
def FinalizeStep(scene=None):
    # Evaluate upper trace of solution for
    #  * for error evaluation
    #  * upwind-coupling to next time slab
    RestrictGFInTime(spacetime_gf=gfu, reference_time=1.0, space_gf=u_last)

    # Compute error at final time
    l2error = sqrt(Integrate((u_exact - u_last)**2 * dOmnew, mesh))
    print("\rt = {0:12.9f}, L2 error = {1:12.9e}".format(told.Get(), l2error), end="")
    if scene:
        scene.Redraw()

### The final time loop

In [31]:
scene = DrawDC(lsetadap.levelsetp1[TOP],u_last,-2, mesh,"u", autoscale=False, min = -2, max = 1, deformation = lsetadap.deformation[TOP])

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

Due to the mesh deformation changing (discontinuously) between time slabs, we have to project solutions from one time slab to the other. This is automatically done for all `GridFunctions` registered in `lsetadap` by `ProjectOnUpdate`.

In [32]:
# Set initial values
u_last.Set(fix_tref(u_exact, 0))
# Project u_last at the beginning of each time step
lsetadap.ProjectOnUpdate(u_last)
told.Set(0)
gf_lset_to_show = GridFunction(H1(mesh, order=1),multidim=0)
gf_u_to_show = GridFunction(u_last.space,multidim=0)
timestep = 0
md_comps = 0
with TaskManager():
    while tend - told.Get() > delta_t/2:
        UpdateTimeSlabGeometry()
        if timestep % 1 == 0:
            gf_lset_to_show.AddMultiDimComponent(lsetadap.levelsetp1[BOTTOM].vec)
            gf_u_to_show.AddMultiDimComponent(u_last.vec)
            md_comps += 1
        SolveForTimeSlab()       
        timestep += 1
        told.Set(told.Get() + delta_t)
        FinalizeStep(scene)
gf_lset_to_show.AddMultiDimComponent(lsetadap.levelsetp1[TOP].vec)
gf_u_to_show.AddMultiDimComponent(u_last.vec)
print("")

t =  0.500000000, L2 error = 3.272294664e-01
t =  0.500000000, L2 error = 3.272294664e-01


In [33]:
from helper import MultiDimPairToGF
gf_pair_to_show = MultiDimPairToGF(gf_lset_to_show,gf_u_to_show,mesh,sampling=md_comps)
Draw(gf_pair_to_show, mesh,"uh",eval_function="value.x>0.0?value.z:value.y",autoscale=False,
     min=-0.5,max=1, interpolate_multidim=True, animate=True)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene